In [145]:
%pip install streamlit
%pip install openai
%pip install PyPDF2





In [146]:
import re # Importa el módulo para expresiones regulares
from datetime import datetime, timedelta # Importa clases para manejar fechas y tiempos
from typing import Dict, List, Any, Optional # Importa tipos para anotaciones de tipo


'''La siguiente celda es la clase subvenciones parser el cual se compone de funciones para detectar parámetros, los cuales varios de ellos responden a nùmeros.
 Con la documentación que nos compartió Carmen, hice mapas de palabras comunes para unirlos con las categorías de cada argumento (por ejemplo actividades, instrumentos,regiones etc),
al final de todo la funcion "parsear_busqueda_subvenciones" es la que toma un texto (query) y devuelve un diccionario con los argumentos detectados para refinar la busqueda de convocatorias.


'''

class SubvencionesParser:
    """
    Clase para parsear texto en lenguaje natural y extraer parámetros de búsqueda
    para subvenciones. Permite identificar regiones, tipos de administración,
    beneficiarios, instrumentos, finalidades, actividades y fechas.
    """
    def __init__(self):
        """
        Constructor de la clase SubvencionesParser.
        Inicializa todos los mapeos (diccionarios) y patrones regex
        necesarios para la extracción de información.
        """
        # --- Mapeos de términos a IDs/valores ---

        # Mapeo de nombres de regiones/provincias de España a sus IDs numéricos.
        # Las claves son strings en minúsculas y los valores son listas que contienen el ID.
        self.regiones_map = {
            "a coruña": [4], "lugo": [5], "ourense": [6], "pontevedra": [7],
            "galicia": [3], "asturias": [9], "principado de asturias": [8],
            "cantabria": [11], "cantabria": [10], # Nota: 'cantabria' aparece dos veces con IDs diferentes, revisar si es intencional
            "noroeste": [2], "araba/álava": [14], "gipuzkoa": [15],
            "bizkaia": [16], "pais vasco": [13], "navarra": [18],
            "comunidad foral de navarra": [17], "la rioja": [20],
            "la rioja": [19], # Nota: 'la rioja' aparece dos veces con IDs diferentes, revisar si es intencional
            "huesca": [22], "teruel": [23], "zaragoza": [24],
            "aragon": [21], "noreste": [12], "madrid": [27],
            "comunidad de madrid": [26], "centro (es)": [28],
            "ávila": [30], "burgos": [31], "león": [32],
            "palencia": [33], "salamanca": [34], "segovia": [35],
            "soria": [36], "valladolid": [37], "zamora": [38],
            "castilla y leon": [29], "albacete": [40], "ciudad real": [41],
            "cuenca": [42], "guadalajara": [43], "toledo": [44],
            "castilla la mancha": [39], "badajoz": [46], "cáceres": [47],
            "extremadura": [45], "barcelona": [50], "girona": [51],
            "lleida": [52], "tarragona": [53], "cataluña": [49],
            "alicante": [55], "castellón": [56], "valencia": [57],
            "comunidad valenciana": [54], "eivissa y formentera": [59],
            "mallorca": [60], "menorca": [61], "illes balears": [58],
            "este": [48], "almería": [64], "cádiz": [65], "córdoba": [66],
            "granada": [67], "huelva": [68], "jaén": [69], "málaga": [70],
            "sevilla": [71], "andalucia": [63], "murcia": [73],
            "region de murcia": [72], "ceuta": [75], "ciudad autonoma de ceuta": [74],
            "melilla": [77], "ciudad autonoma de melilla": [76],
            "sur": [62], "el hierro": [80], "fuerteventura": [81],
            "gran canaria": [82], "la gomera": [83], "la palma": [84],
            "lanzarote": [85], "tenerife": [86], "canarias": [79],
            "españa": [1], "extra-regio nuts 1": [87]
        }

        # Mapeo de términos de tipo de administración a códigos (C: Central, A: Autonómica, L: Local, O: Otros).
        self.tipos_administracion_map = {
            'estatal': 'C', 'estado': 'C', 'central': 'C', 'gobierno': 'C', 'ministerio': 'C',
            'autonómica': 'A', 'autonomica': 'A', 'comunidad autónoma': 'A', 'comunidad autonoma': 'A',
            'local': 'L', 'ayuntamiento': 'L', 'municipio': 'L', 'diputación': 'L', 'diputacion': 'L',
            'otros': 'O', 'otras': 'O', 'otro': 'O', 'organismo': 'O'
        }

        # Mapeo de palabras clave de tipos de beneficiario a sus IDs numéricos.
        # Los IDs están en listas, lo que permite flexibilidad para futuras expansiones.
        self.tipos_beneficiario_map = {
            "pyme": [3], "autonomo": [3], "empresario individual": [3], "profesional": [3],
            "gran empresa": [4], "empresa grande": [4], "sociedad anonima": [4], "sociedad limitada": [4],
            "asociacion": [2], "fundacion": [2], "ong": [2], "club deportivo": [2], "entidad sin animo de lucro": [2],
            "particular": [1], "persona fisica": [1], "ciudadano": [1], "individuo": [1],
            "sin informacion": [5], "desconocido": [5]
        }

        # Mapeo de palabras clave de actividades económicas a sus IDs numéricos.
        # Las claves son strings en minúsculas.
        self.actividad_map = {
            "agricultura": 274, "ganaderia": 274, "silvicultura": 274, "pesca": 274,
            "industrias extractivas": 278, "mineria": 278,
            "manufacturera": 284, "industria manufacturera": 284, "fabricacion": 284,
            "suministro energia": 309, "electricidad": 309, "gas": 309, "vapor": 309, "aire acondicionado": 309,
            "suministro agua": 311, "saneamiento": 311, "residuos": 311, "gestion residuos": 311, "descontaminacion": 311,
            "construccion": 316,
            "comercio": 320, "comercio mayor": 320, "comercio menor": 320, "reparacion vehiculos": 320, "reparacion motos": 320,
            "transporte": 324, "almacenamiento": 324,
            "hosteleria": 330, "restaurantes": 330, "hoteles": 330, "bares": 330,
            "informacion": 333, "comunicaciones": 333, "telecomunicaciones": 333, "informatica": 333,
            "actividades financieras": 340, "seguros": 340, "banca": 340,
            "inmobiliarias": 344, "actividades inmobiliarias": 344,
            "profesionales": 346, "cientificas": 346, "tecnicas": 346, "ingenieria": 346, "consultoria": 346, "juridicas": 346, "contabilidad": 346,
            "administrativas": 354, "servicios auxiliares": 354, "oficina": 354, "seguridad privada": 354, "limpieza": 354,
            "administracion publica": 361, "defensa": 361, "seguridad social": 361,
            "educacion": 363,
            "sanitarias": 365, "servicios sociales": 365, "salud": 365, "hospitales": 365, "residencias": 365,
            "artisticas": 369, "recreativas": 369, "entretenimiento": 369, "ocio": 369, "deporte": 369,
            "otros servicios": 374, "servicios personales": 374,
            "hogares empleadores": 378, "servicio domestico": 378, "uso propio": 378,
            "extraterritoriales": 381, "organizaciones extraterritoriales": 381
        }

        # Mapeo de palabras clave de instrumentos de ayuda a sus IDs numéricos.
        # Los IDs están en listas, permitiendo asociar múltiples palabras a un mismo ID.
        self.instrumentos_map = {
            "subvencion": [1], "ayuda": [1], "ayudas":[1], "becas":[1], "dinero": [1],
            "prestamo": [2], "credito": [2], "financiacion": [2],
            "garantia": [4], "aval": [4],
            "ventaja fiscal": [5], "deduccion": [5], "impuesto": [5], "fiscal": [5], "exencion": [5],
            "aportacion riesgo": [6], "capital riesgo": [6]
        }

        # Mapeo de palabras clave de finalidades de política de gasto a sus IDs numéricos.
        self.finalidades_map = {
            "vivienda": 8, "edificacion": 8,
            "agricultura": 12, "pesca": 12, "alimentacion": 12, "alimentos": 12,
            "comercio": 14, "turismo": 14, "pymes": 14,
            "desarrollo": 20, "cooperacion": 20, "cultural": 20,
            "cultura": 11, "defensa": 2,
            "desempleo": 7, "parado": 7, "paro": 7,
            "educacion": 10, "estudios": 10, "escuela": 10, "universidad": 10,
            "empleo": 6, "trabajo": 6, "fomento empleo": 6,
            "industria": 13, "energia": 13, "electricidad": 13, "gas": 13,
            "informacion no disponible": 21,
            "infraestructuras": 16, "carreteras": 16, "puertos": 16, "aeropuertos": 16, "tren": 16,
            "investigacion": 17, "innovacion": 17, # 'desarrollo' se repite, pero aquí tiene el ID 17
            "justicia": 1, "juzgados": 1, "ley": 1,
            "economico": 18, "otras economicas": 18,
            "prestaciones economicas": 4,
            "sanidad": 9, "salud": 9, "hospitales": 9, "medico": 9,
            "seguridad ciudadana": 3, "prisiones": 3, "carcel": 3, "policia": 3,
            "servicios sociales": 5, "sociales": 5, "promocion social": 5,
            "sin informacion": 19,
            "transporte": 15, "subvenciones transporte": 15
        }

        # --- Patrones de Expresiones Regulares para Fechas ---
        self.fecha_patterns = [
            # dd/mm/yyyy o dd-mm-yyyy precedido de 'desde' o 'a partir de'
            r'desde\s+(\d{1,2})[/-](\d{1,2})[/-](\d{4})',
            r'a partir del?\s+(\d{1,2})[/-](\d{1,2})[/-](\d{4})',
            # dd/mm/yyyy o dd-mm-yyyy precedido de 'hasta' o 'antes de'
            r'hasta\s+(\d{1,2})[/-](\d{1,2})[/-](\d{4})',
            r'antes del?\s+(\d{1,2})[/-](\d{1,2})[/-](\d{4})',
            # Rango de fechas: dd/mm/yyyy hasta dd/mm/yyyy
            r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\s+hasta\s+(\d{1,2})[/-](\d{1,2})[/-](\d{4})',
            # Año solo (ej. '2025'). \b asegura que sea una palabra completa para evitar coincidencias parciales.
            r'\b(\d{4})\b'
        ]

    def extraer_parametros(self, texto: str) -> Dict[str, Any]:
        """
        Función principal para extraer parámetros de búsqueda de subvenciones
        desde texto en lenguaje natural. Procesa el texto y llama a funciones
        auxiliares para identificar diferentes tipos de información.

        Args:
            texto (str): Texto de búsqueda en lenguaje natural proporcionado por el usuario.

        Returns:
            Dict[str, Any]: Un diccionario que contiene todos los parámetros
                            extraídos, listos para ser utilizados en una API
                            o sistema de búsqueda de subvenciones.
        """
        # Normaliza el texto de entrada: convierte a minúsculas y elimina espacios en blanco al inicio/final.
        texto_lower = texto.lower().strip()
        parametros = {} # Inicializa un diccionario vacío para almacenar los parámetros encontrados.

        # --- Extracción de la Descripción (términos clave) ---
        # Llama a la función auxiliar para extraer los términos de búsqueda principales.
        descripcion = self._extraer_descripcion(texto_lower)
        if descripcion:
            parametros['descripcion'] = descripcion
            # Establece un tipo de búsqueda por defecto si se encuentra una descripción.
            parametros['descripcionTipoBusqueda'] = 2  # '2' suele significar "todas las palabras"

        # --- Extracción de Regiones ---
        # Llama a la función auxiliar para identificar regiones.
        regiones = self._extraer_regiones(texto_lower)
        if regiones:
            parametros['regiones'] = regiones

        # --- Extracción del Tipo de Administración ---
        # Llama a la función auxiliar para determinar si la búsqueda es estatal, autonómica, local, etc.
        tipo_admin = self._extraer_tipo_administracion(texto_lower)
        if tipo_admin:
            parametros['tipoAdministracion'] = tipo_admin

        # --- Extracción de Tipos de Beneficiario ---
        # Llama a la función auxiliar para identificar a quién va dirigida la subvención.
        tipos_benef = self._extraer_tipos_beneficiario(texto_lower)
        if tipos_benef:
            parametros['tiposBeneficiario'] = tipos_benef

        # --- Extracción de Instrumentos ---
        # Llama a la función auxiliar para identificar el tipo de ayuda (subvención, préstamo, etc.).
        instrumentos = self._extraer_instrumentos(texto_lower)
        if instrumentos:
            parametros['instrumentos'] = instrumentos

        # --- Extracción de Finalidad ---
        # Llama a la función auxiliar para determinar la finalidad o sector de la política de gasto.
        finalidad = self._extraer_finalidad(texto_lower)
        if finalidad:
            parametros['finalidad'] = finalidad

        # --- Extracción de Actividad Económica ---
        # Llama a la función auxiliar para identificar el tipo de actividad económica.
        actividad = self._extraer_actividad(texto_lower)
        if actividad:
            parametros['actividad'] = actividad

        # --- Extracción de Fechas ---
        # Llama a la función auxiliar para identificar fechas o rangos de fechas.
        fechas = self._extraer_fechas(texto_lower)
        if fechas:
            # Agrega los pares clave-valor de fechas al diccionario de parámetros principal.
            parametros.update(fechas)

        # --- Extracción de Número de Convocatoria ---
        # Llama a la función auxiliar para buscar un número de convocatoria específico (ej. BDNS).
        numero_conv = self._extraer_numero_convocatoria(texto) # Usa el texto original, no el lower para posibles mayúsculas en códigos
        if numero_conv:
            parametros['numeroConvocatoria'] = numero_conv

        # --- Detección de MRR (Mecanismo de Recuperación y Resiliencia) ---
        # Busca términos relacionados con los fondos Next Generation.
        if any(term in texto_lower for term in ['mrr', 'recuperación', 'resiliencia', 'next generation']):
            parametros['mrr'] = True

        # --- Configuración de Parámetros por Defecto ---
        # Establece valores por defecto para parámetros que no fueron especificados en la consulta.
        if 'descripcionTipoBusqueda' not in parametros:
            parametros['descripcionTipoBusqueda'] = 2  # Por defecto: buscar todas las palabras de la descripción.
        if 'order' not in parametros:
            parametros['order'] = 'fechaRecepcion' # Ordenar por fecha de recepción.
        if 'direccion' not in parametros:
            parametros['direccion'] = 'desc'  # Orden descendente (más recientes primero).
        if 'vpd'not in parametros:
            parametros['vpd'] = 'GE' # Valor por defecto para 'vpd' (posiblemente 'Vigencia de Publicación/Documento').
        if 'page' not in parametros:
            parametros['page'] = 0 # Página inicial de resultados.
        if 'pageSize' not in parametros:
            parametros['pageSize'] = 25 # Número de resultados por página.

        return parametros

    def _extraer_descripcion(self, texto: str) -> Optional[str]:
        """
        Extrae los términos clave de la descripción de la búsqueda,
        filtrando palabras comunes (stop words) que no aportan significado.

        Args:
            texto (str): Texto de la consulta en minúsculas.

        Returns:
            Optional[str]: Una cadena con las palabras clave extraídas (máximo 5),
                           o None si no se encuentran palabras clave significativas.
        """
        # Palabras a ignorar que son muy comunes o funcionales en las consultas.
        stop_words = {'ayudame', 'ayúdame', 'buscar', 'encontrar',
                     'para', 'de', 'en', 'el', 'la', 'los', 'las', 'un', 'una', 'referentes',
                     'sobre', 'acerca', 'relacionado', 'relacionados', 'me', 'que', 'quiero'}

        # Encuentra todas las palabras alfanuméricas en el texto.
        palabras = re.findall(r'\b\w+\b', texto)
        # Filtra las palabras: no deben ser stop words y deben tener más de 2 caracteres.
        palabras_clave = [p for p in palabras if p.lower() not in stop_words and len(p) > 2]

        if palabras_clave:
            # Retorna un máximo de 5 palabras clave, unidas por espacios.
            return ' '.join(palabras_clave[:5])
        return None

    def _extraer_regiones(self, texto: str) -> Optional[List[int]]:
        """
        Extrae los IDs de las regiones españolas mencionadas en el texto.

        Args:
            texto (str): Texto de la consulta en minúsculas.

        Returns:
            Optional[List[int]]: Una lista de IDs únicos de regiones encontradas,
                                  o None si no se identifica ninguna región.
        """
        regiones_encontradas = []
        # Itera sobre el mapeo de regiones.
        for region, ids in self.regiones_map.items():
            # Si el nombre de la región está en el texto, añade sus IDs a la lista.
            if region in texto:
                regiones_encontradas.extend(ids)

        # Devuelve una lista de IDs únicos, o None si la lista está vacía.
        return list(set(regiones_encontradas)) if regiones_encontradas else None

    def _extraer_tipo_administracion(self, texto: str) -> Optional[str]:
        """
        Extrae el tipo de administración (Estatal, Autonómica, Local, Otros)
        mencionado en el texto.

        Args:
            texto (str): Texto de la consulta en minúsculas.

        Returns:
            Optional[str]: El código del tipo de administración ('C', 'A', 'L', 'O'),
                           o None si no se identifica.
        """
        # Itera sobre el mapeo de tipos de administración.
        for tipo, codigo in self.tipos_administracion_map.items():
            # Si el término del tipo de administración está en el texto, retorna su código.
            # Se detiene en la primera coincidencia.
            if tipo in texto:
                return codigo
        return None

    def _extraer_tipos_beneficiario(self, texto: str) -> Optional[List[int]]:
        """
        Extrae los IDs de los tipos de beneficiario mencionados en el texto.

        Args:
            texto (str): Texto de la consulta en minúsculas.

        Returns:
            Optional[List[int]]: Una lista de IDs únicos de beneficiarios encontrados,
                                  o None si no se identifica ninguno.
        """
        tipos_encontrados = []
        # Itera sobre el mapeo de tipos de beneficiario.
        for tipo, ids in self.tipos_beneficiario_map.items():
            # Si el término del tipo de beneficiario está en el texto, añade sus IDs.
            if tipo in texto:
                tipos_encontrados.extend(ids)

        # Devuelve una lista de IDs únicos, o None si la lista está vacía.
        return list(set(tipos_encontrados)) if tipos_encontrados else None

    def _extraer_instrumentos(self, texto: str) -> Optional[List[int]]:
        """
        Extrae los IDs de los instrumentos de ayuda (ej. subvención, préstamo)
        mencionados en el texto.

        Args:
            texto (str): Texto de la consulta en minúsculas.

        Returns:
            Optional[List[int]]: Una lista de IDs únicos de instrumentos encontrados,
                                  o None si no se identifica ninguno.
        """
        instrumentos_encontrados = []
        # Itera sobre el mapeo de instrumentos.
        for instrumento, ids in self.instrumentos_map.items():
            # Si el término del instrumento está en el texto, añade sus IDs.
            if instrumento in texto:
                instrumentos_encontrados.extend(ids)

        # Devuelve una lista de IDs únicos, o None si la lista está vacía.
        return list(set(instrumentos_encontrados)) if instrumentos_encontrados else None

    def _extraer_finalidad(self, texto: str) -> Optional[int]:
        """
        Extrae el ID de la finalidad de la política de gasto (ej. vivienda, agricultura)
        mencionada en el texto.

        Args:
            texto (str): Texto de la consulta en minúsculas.

        Returns:
            Optional[int]: El ID de la finalidad encontrada, o None si no se identifica.
                           Retorna la primera coincidencia.
        """
        # Itera sobre el mapeo de finalidades.
        for finalidad, codigo in self.finalidades_map.items():
            # Si el término de la finalidad está en el texto, retorna su ID.
            # Se detiene en la primera coincidencia.
            if finalidad in texto:
                return codigo
        return None

    def _extraer_actividad(self, texto: str) -> Optional[int]:
        """
        Extrae el ID de la actividad económica mencionada en el texto.

        Args:
            texto (str): Texto de la consulta en minúsculas.

        Returns:
            Optional[int]: El ID de la actividad encontrada, o None si no se identifica.
                           Retorna la primera coincidencia.
        """
        # Itera sobre el mapeo de actividades.
        for actividad, codigo in self.actividad_map.items():
            # Si el término de la actividad está en el texto, retorna su ID.
            # Se detiene en la primera coincidencia.
            if actividad in texto:
                return codigo
        return None

    def _extraer_fechas(self, texto: str) -> Dict[str, str]:
        """
        Extrae fechas o rangos de fechas (fechaDesde, fechaHasta, anioInteres)
        del texto utilizando expresiones regulares.

        Args:
            texto (str): Texto de la consulta en minúsculas.

        Returns:
            Dict[str, str]: Un diccionario con las fechas extraídas .
        """
        fechas = {}
        # Itera sobre cada patrón de expresión regular predefinido para fechas.
        for pattern in self.fecha_patterns:
            # Encuentra todas las coincidencias del patrón en el texto.
            matches = re.finditer(pattern, texto)
            for match in matches:
                # Lógica para patrones de "desde" o "a partir de" (formato dd/mm/yyyy)
                if 'desde' in pattern or 'partir' in pattern:
                    # Verifica que haya al menos 3 grupos capturados (día, mes, año)
                    if len(match.groups()) >= 3:
                        # Formatea la fecha y la asigna a 'fechaDesde'. zfill(2) añade un cero inicial si es necesario (ej. 1 -> 01).
                        fechas['fechaDesde'] = f"{match.group(1).zfill(2)}/{match.group(2).zfill(2)}/{match.group(3)}"
                # Lógica para patrones de "hasta" o "antes de" (formato dd/mm/yyyy)
                elif 'hasta' in pattern or 'antes' in pattern:
                    if len(match.groups()) >= 3:
                        fechas['fechaHasta'] = f"{match.group(1).zfill(2)}/{match.group(2).zfill(2)}/{match.group(3)}"
                # Lógica para el patrón de rango completo (dd/mm/yyyy hasta dd/mm/yyyy)
                elif len(match.groups()) == 6: # Este patrón captura 6 grupos (3 para la primera fecha, 3 para la segunda)
                    fechas['fechaDesde'] = f"{match.group(1).zfill(2)}/{match.group(2).zfill(2)}/{match.group(3)}"
                    fechas['fechaHasta'] = f"{match.group(4).zfill(2)}/{match.group(5).zfill(2)}/{match.group(6)}"

        return fechas

    def _extraer_numero_convocatoria(self, texto: str) -> Optional[str]:
        """
        Extrae un posible número de convocatoria BDNS (Base de Datos Nacional de Subvenciones),
        identificando una secuencia de 6 dígitos como palabra completa.

        Args:
            texto (str): Texto original de la consulta.

        Returns:
            Optional[str]: El número de convocatoria encontrado como string, o None si no se halla.
        """
        # Busca una secuencia de exactamente 6 dígitos rodeada por límites de palabra.
        match = re.search(r'\b\d{6}\b', texto)
        return match.group() if match else None


# --- Función Principal de Parsing ---
def parsear_busqueda_subvenciones(texto_busqueda: str) -> Dict[str, Any]:
    """
    Función principal de utilidad que encapsula la lógica de parsing de búsquedas
    de subvenciones. Crea una instancia de SubvencionesParser y llama a su
    método `extraer_parametros`.

    Args:
        texto_busqueda (str): El texto de la búsqueda en lenguaje natural del usuario.

    Returns:
        Dict[str, Any]: Un diccionario con los parámetros de búsqueda extraídos.
    """
    parser = SubvencionesParser()
    return parser.extraer_parametros(texto_busqueda)

In [147]:
from logging import warning
import streamlit as st
import requests
import pandas as pd
import re
from datetime import datetime
import openai
import json
import os
import unicodedata
import PyPDF2
import time



# ---------- FUNCIONES AUXILIARES -----------


'''Aqui hay algunas funciones auxiliares basadas en el trabajo de Carmen para comunicarnos con el sitio web de subvenciones.

'''

def buscar_convocatorias(**params): #esta funcion toma el diccionario de palabras obtenido de la función "parsear_busqueda_subvenciones" y hace un request con la API al sitio de subvenciones.
#se obtiene un DF con las convocatorias que satisfacen los argumentos del diccionario.
    base_url = "https://www.pap.hacienda.gob.es/bdnstrans/api/convocatorias/busqueda"
    headers = {"Accept": "application/json"}
    page_size = params.get("pageSize", 25)
    max_paginas = 3
    resultados = []
    params.setdefault("vpd", "GE")
    params.setdefault("pageSize", page_size)
    for pagina in range(0, max_paginas):
        params["page"] = pagina
        try:
            response = requests.get(base_url, params=params, headers=headers)
            response.raise_for_status()
            if "application/json" in response.headers.get("Content-Type", ""):
                data = response.json()
                convocatorias = data.get("convocatorias", data.get("content", []))
                if not convocatorias:
                    break
                resultados.extend(convocatorias)
                time.sleep(0.5)
        except Exception as e:
            break
    return pd.DataFrame(resultados)

def obtener_convocatoria_por_id(num_conv): #esta funcion toma un numero especifico de convocatoria y realiza un request a la API, la cual devuelve un diccionario con información específica de la convocatoria.
    base_url = "https://www.infosubvenciones.es/bdnstrans/api/convocatorias"
    params = {"vpd": "GE", "numConv": str(num_conv)}
    headers = {"Accept": "application/json"}
    r = requests.get(base_url, params=params, headers=headers)
    if r.status_code == 200 and "application/json" in r.headers.get("Content-Type", ""):
        return r.json()
    else:
        return None

def descargar_documento_pdf(documento_id, nombre): #esta funcion requiere el documento_id y nombre y descarga en ruta relativa el pdf en cuestión.
    url = f"https://www.infosubvenciones.es/bdnstrans/api/convocatorias/documentos?idDocumento={documento_id}"
    respuesta = requests.get(url)
    if respuesta.status_code == 200:
        ruta_relativa = os.path.join("documentos", nombre)
        ruta_absoluta = os.path.abspath(ruta_relativa)
        os.makedirs(os.path.dirname(ruta_absoluta), exist_ok=True)
        with open(ruta_absoluta, "wb") as f:
            f.write(respuesta.content)
        return ruta_absoluta
    return None

def mostrar_resumen_json(convocatoria): #es una funcion que hice para que al hacer clic sobre una convocatoria, se muestre un resumen pequeño de la convocatoria, antes de dar la opción a ver mas detalles . (Solo para demo de streamlit)
    resumen = []
    resumen.append(f"**Presupuesto total:** {convocatoria.get('presupuestoTotal', 'N/D')} €")
    resumen.append(f"**Fechas:** del {convocatoria.get('fechaInicioSolicitud', '¿?')} al {convocatoria.get('fechaFinSolicitud', '¿?')}")
    resumen.append(f"**Finalidad:** {convocatoria.get('descripcionFinalidad', 'No especificada')}")
    tipos = convocatoria.get("tiposBeneficiarios", [])
    if tipos:
        resumen.append(f"**Beneficiarios:** {', '.join(t.get('descripcion', '') for t in tipos)}")
    resumen.append(f"**Abierta:** {'✅ Sí' if convocatoria.get('abierto') else '❌ No'}")
    return "\n".join(resumen)


# ---------- INTERFAZ STREAMLIT -----------


st.title("🔍 Buscador de convocatorias públicas")

query = st.text_input("¿Qué convocatorias buscas?", key="query_input_main")

if query:
    try:
        filtros = parsear_busqueda_subvenciones(query)
        with st.spinner("Buscando convocatorias, por favor espera..."):
            df = buscar_convocatorias(**filtros)

        if not df.empty:
            st.success(f"Se encontraron {len(df)} convocatorias.")
             # --- NUEVO PASO CRUCIAL ---
            # Convertir la columna 'fechaPublicacion' a datetime
            # Primero, aseguramos que la columna exista para evitar errores si la API no la devuelve
            if 'fechaRecepcion' in df.columns:
                # Usamos errors='coerce' para convertir a NaT (Not a Time) si hay valores inválidos
                df['fechaRecepcion'] = pd.to_datetime(df['fechaRecepcion'], errors='coerce')

            # Añadir una columna booleana para los checkboxes
            # Inicialmente, ninguna está seleccionada
            df['Seleccionar'] = False

            # Columnas a mostrar y renombrar
            display_df = df[['Seleccionar', 'numeroConvocatoria', 'descripcion', 'fechaRecepcion', 'nivel1']].copy()
            display_df.columns = ['Seleccionar', 'Nº Convocatoria', 'Descripción', 'Fecha Publicación', 'Nivel1*']

            st.write("### Convocatorias encontradas:")

            # Usar st.data_editor con CheckboxColumn
            edited_df = st.data_editor(
                display_df,
                column_config={
                    "Seleccionar": st.column_config.CheckboxColumn(
                        "Seleccionar",
                        help="Marca para ver el resumen de la convocatoria",
                        default=False,
                        # Si quieres que solo se pueda seleccionar una, la lógica es un poco más avanzada
                        # y requiere manejar el estado para deseleccionar otras.
                        # Para selección múltiple, esto funciona directamente.
                    ),
                    "Descripción": st.column_config.TextColumn(
                        "Descripción",
                        help="Descripción de la convocatoria",
                        width="large", # Para que la descripción ocupe más espacio
                    ),
                    "Nº Convocatoria": st.column_config.TextColumn("Nº Convocatoria", width="small"),
                    "Cuantía": st.column_config.TextColumn("Cuantía", width="small"),
                    "Fecha Publicación": st.column_config.DateColumn("Fecha Publicación", width="small"),
                },
                hide_index=True,
                num_rows="fixed", # No permitir añadir/eliminar filas
                use_container_width=True, # Usar el ancho completo de la columna/página
                key="convocatorias_editor"
            )

            # --- Lógica para mostrar el resumen de la convocatoria seleccionada ---
            # Filtrar las filas que tienen el checkbox marcado
            selected_rows = edited_df[edited_df['Seleccionar'] == True]

            if not selected_rows.empty:
                # Si hay múltiples seleccionadas, puedes elegir mostrar la primera,
                # o mostrar un resumen para cada una, o pedir al usuario que deseleccione.
                # Para el ejemplo, mostraremos la primera seleccionada.
                selected_convocatoria = selected_rows.iloc[0]
                num_conv = selected_convocatoria['Nº Convocatoria']

                # Guardar en session_state para mantener el estado si se refresca la página
                st.session_state.num_conv_selected = num_conv

                st.divider() # Un separador visual
                st.subheader(f"Resumen de la convocatoria {num_conv}")

                try:
                    with st.spinner(f"Cargando detalles de {num_conv}..."):
                        data = obtener_convocatoria_por_id(num_conv)

                    if data:
                        st.markdown(mostrar_resumen_json(data))
                    else:
                        st.warning(f"No se pudieron obtener los detalles para la convocatoria {num_conv}.")
                except Exception as e_detail:
                    st.error(f"⚠️ Error al cargar detalles: {e_detail}")
            else:
                st.info("Marca el checkbox de una convocatoria en la tabla para ver su resumen.")

        else:
            st.warning("No se encontraron convocatorias que cumplan con los criterios de búsqueda.")

    except Exception as e:
        # Este except general captura cualquier excepción lanzada por
        # parsear_busqueda_subvenciones o buscar_convocatorias.
        st.error(f"🚫 ¡Ha ocurrido un problema! Por favor, intenta una búsqueda diferente o verifica tu conexión. Detalles: {e}")





2025-07-01 19:11:21.542 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-01 19:11:21.542 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-01 19:11:21.544 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-01 19:11:21.545 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-01 19:11:21.546 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-01 19:11:21.547 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-01 19:11:21.548 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-01 19:11:21.550 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [150]:
#ejemplo de funcionamiento en codigo:

query='ayudas vivienda madrid'


filtros=parsear_busqueda_subvenciones(query)
convocatoria= buscar_convocatorias(**filtros)
convocatoria

,id,mrr,numeroConvocatoria,descripcion,descripcionLeng,fechaRecepcion,nivel1,nivel2,nivel3,codigoInvente
0,1036319,False,834758,Convocatoria pública de subvenciones con desti...,None,2025-05-23,MADRID,ÁREA DE GOBIERNO DE POLÍTICAS DE VIVIENDA,None,None
1,1025183,False,823622,Convocatoria pública subvenciones 2025 con des...,None,2025-03-28,MADRID,ÁREA DE GOBIERNO DE POLÍTICAS DE VIVIENDA,None,None
2,994929,False,793369,Decreto de 24 de Octubre de 2024 del Delegado ...,None,2024-10-25,MADRID,ÁREA DE GOBIERNO DE POLÍTICAS DE VIVIENDA,None,None
3,994922,False,793362,Decreto de 24 de Octubre de 2024 del Delegado ...,None,2024-10-25,MADRID,ÁREA DE GOBIERNO DE POLÍTICAS DE VIVIENDA,None,None
4,994915,False,793355,Decreto de 24 de Octubre de 2024 del Delegado ...,None,2024-10-25,MADRID,ÁREA DE GOBIERNO DE POLÍTICAS DE VIVIENDA,None,None
5,988613,False,787053,convocatoria pública de subvenciones con desti...,None,2024-09-24,MADRID,ÁREA DE GOBIERNO DE POLÍTICAS DE VIVIENDA,None,None
6,954677,False,753117,Convocatoria pública de subvenciones 2024 con ...,None,2024-04-05,MADRID,ÁREA DE GOBIERNO DE POLÍTICAS DE VIVIENDA,None,None
7,937298,False,735738,convocatoria pública de subvenciones 2022 con...,None,2023-12-27,MADRID,ÁREA DE GOBIERNO DE POLÍTICAS DE VIVIENDA,None,None
8,920764,False,719204,Decreto del Delegado del Área de Gobierno de D...,None,2023-10-02,MADRID,ÁREA DE GOBIERNO DE POLÍTICAS DE VIVIENDA,None,None
9,919432,False,717872,convocatoria pública de subvenciones 2023 con ...,None,2023-09-22,MADRID,ÁREA DE GOBIERNO DE POLÍTICAS DE VIVIENDA,None,None
